Import ellis_joined, from kernel in preceeding notebook which details the cleaning and joing process.

We are now going to use another assessor's dataset from 2019, and concern ourselves with the period of 2014-2019 in particuar in order to study the determinants of the ellis act on the property level during this period of intensive use.

Datasets:

Ellis Act Filing Records 2007- Present (May 2021): https://hcidla2.lacity.org/ELLIS - base dataset, now joined, see previous notebook - we will be extracting data from years 2015-2019 for this study. 

Census tract level demographic and econmoic data: ACS 5 Years 2014 and 2019 from census api through cenpy. Index of tables https://www2.census.gov/programs-surveys/acs/tech_docs/table_shells/2019/: data for Los Angeles County Census Tracts.

Los Angeles County Assessors Parcel Records (2019): Purchased by SAJE, used with permissions - used to identify parcel status, features and ownership - GEOJSON export including selected columns for LA city parcels from SAJE owned geopackage.

In [1]:
#Import modules: pandas/geopandas
import pandas as pd
import geopandas as gpd

In [2]:
ellisdf=pd.read_csv('/Users/alexanderferrer/Documents/GitHub/UP229_Ellis/ellis_joined.csv')

ellisdf.head()

,Unnamed: 0,application_received,application_year,council_district,address,city,zip,units_withdrawn,replacement_unit,yearbuilt,units,sqftmain,center_lat,center_lon,geometry
0,2040031029,11/6/19,2019,3,22729 W DEL VALLE ST,LOS ANGELES,91364.0,3,0.0,1955.0,3.0,1860.0,34.165880,-118.621319,POINT (-13204864.81828771 4051097.450097568)
1,2040031030,12/4/19,2019,3,22727 W DEL VALLE ST,WOODLAND HILLS,91364.0,3,0.0,1955.0,3.0,1869.0,34.165929,-118.621164,POINT (-13204847.61386041 4051104.006178072)
2,2040033033,2/10/16,2016,3,22431 W DEL VALLE ST,WOODLAND HILLS,91364.0,3,0.0,1955.0,3.0,1869.0,34.167389,-118.616525,POINT (-13204331.1860447 4051300.523521864)
3,2107014004,4/21/20,2020,3,20327 W STAGG ST,LOS ANGELES,91306.0,3,0.0,1910.0,3.0,2446.0,34.211389,-118.576350,POINT (-13199858.95667153 4057221.803746735)
4,2111025021,3/16/16,2016,3,7118 N JORDAN AVE,CANOGA PARK,91303.0,2,9.0,1947.0,2.0,1489.0,34.199760,-118.602505,POINT (-13202770.5034817 4055656.567431433)


In [3]:
ellisdf.rename(columns={'Unnamed: 0': 'ain',}, inplace=True)

ellisdf.head()

,ain,application_received,application_year,council_district,address,city,zip,units_withdrawn,replacement_unit,yearbuilt,units,sqftmain,center_lat,center_lon,geometry
0,2040031029,11/6/19,2019,3,22729 W DEL VALLE ST,LOS ANGELES,91364.0,3,0.0,1955.0,3.0,1860.0,34.165880,-118.621319,POINT (-13204864.81828771 4051097.450097568)
1,2040031030,12/4/19,2019,3,22727 W DEL VALLE ST,WOODLAND HILLS,91364.0,3,0.0,1955.0,3.0,1869.0,34.165929,-118.621164,POINT (-13204847.61386041 4051104.006178072)
2,2040033033,2/10/16,2016,3,22431 W DEL VALLE ST,WOODLAND HILLS,91364.0,3,0.0,1955.0,3.0,1869.0,34.167389,-118.616525,POINT (-13204331.1860447 4051300.523521864)
3,2107014004,4/21/20,2020,3,20327 W STAGG ST,LOS ANGELES,91306.0,3,0.0,1910.0,3.0,2446.0,34.211389,-118.576350,POINT (-13199858.95667153 4057221.803746735)
4,2111025021,3/16/16,2016,3,7118 N JORDAN AVE,CANOGA PARK,91303.0,2,9.0,1947.0,2.0,1489.0,34.199760,-118.602505,POINT (-13202770.5034817 4055656.567431433)


In [4]:
ellisdf['application_date'] = pd.to_datetime(ellisdf['application_received'])

print(ellisdf['application_date'].unique())

['2019-11-06T00:00:00.000000000' '2019-12-04T00:00:00.000000000'
 '2016-02-10T00:00:00.000000000' ... '2020-07-20T00:00:00.000000000'
 '2018-02-13T00:00:00.000000000' '2017-01-12T00:00:00.000000000']


In this case we are only interested in Ellis Act Evictions from 2014 to 2019, so we will create a new dataset dropping the others.

In [5]:
studyellisdf=ellisdf[ellisdf['application_year'].between(2015, 2019)]

print(studyellisdf['application_year'].unique())

[2019 2016 2015 2018 2017]


In [6]:
#studyellisdf.to_csv("studyellisdf.csv")

In [7]:
parcelsdf=pd.read_csv('/Users/alexanderferrer/Desktop/assessor_2019_la.csv')

parcelsdf.columns

/Applications/anaconda3/envs/UP299/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['fid', 'CENTER_LON', 'YearBuilt5', 'YearBuilt4', 'YearBuilt1',
       'UseType', 'YearBuilt2', 'APN', 'Bedrooms1', 'Bedrooms2', 'Bedrooms3',
       'Bedrooms4', 'Bedrooms5', 'UseDescription', 'MailAddress', 'SQFTmain2',
       'SQFTmain3', 'SQFTmain1', 'SQFTmain4', 'SQFTmain5', 'MailCity',
       'Units1', 'Units3', 'Units2', 'Units5', 'Units4', 'OwnerFullName',
       'AgencyName', 'AIN', 'Bathrooms5', 'Bathrooms4', 'Bathrooms1',
       'Bathrooms3', 'Bathrooms2', 'SitusFullAddress', 'Shape.STArea()',
       'Shape.STLength()', 'YearBuilt3', 'CENTER_LAT', 'LS1 Sale Amount',
       'LS1 Sale Date', 'LS2 Sale Amount', 'LS2 Sale Date'],
      dtype='object')

In [8]:
parcelsdf.columns=parcelsdf.columns.str.lower()

parcelsdf.columns

Index(['fid', 'center_lon', 'yearbuilt5', 'yearbuilt4', 'yearbuilt1',
       'usetype', 'yearbuilt2', 'apn', 'bedrooms1', 'bedrooms2', 'bedrooms3',
       'bedrooms4', 'bedrooms5', 'usedescription', 'mailaddress', 'sqftmain2',
       'sqftmain3', 'sqftmain1', 'sqftmain4', 'sqftmain5', 'mailcity',
       'units1', 'units3', 'units2', 'units5', 'units4', 'ownerfullname',
       'agencyname', 'ain', 'bathrooms5', 'bathrooms4', 'bathrooms1',
       'bathrooms3', 'bathrooms2', 'situsfulladdress', 'shape.starea()',
       'shape.stlength()', 'yearbuilt3', 'center_lat', 'ls1 sale amount',
       'ls1 sale date', 'ls2 sale amount', 'ls2 sale date'],
      dtype='object')

In [9]:
parcelsdf.columns = parcelsdf.columns.str.replace(' ','_')

parcelsdf.columns

Index(['fid', 'center_lon', 'yearbuilt5', 'yearbuilt4', 'yearbuilt1',
       'usetype', 'yearbuilt2', 'apn', 'bedrooms1', 'bedrooms2', 'bedrooms3',
       'bedrooms4', 'bedrooms5', 'usedescription', 'mailaddress', 'sqftmain2',
       'sqftmain3', 'sqftmain1', 'sqftmain4', 'sqftmain5', 'mailcity',
       'units1', 'units3', 'units2', 'units5', 'units4', 'ownerfullname',
       'agencyname', 'ain', 'bathrooms5', 'bathrooms4', 'bathrooms1',
       'bathrooms3', 'bathrooms2', 'situsfulladdress', 'shape.starea()',
       'shape.stlength()', 'yearbuilt3', 'center_lat', 'ls1_sale_amount',
       'ls1_sale_date', 'ls2_sale_amount', 'ls2_sale_date'],
      dtype='object')

We need to find total values for the parcels' characteristics, so first we need to convert the fields to in and nulls to zeros.

In [10]:
parcelsdf[['bedrooms1', 'bedrooms2', 'bedrooms3','bedrooms4', 'bedrooms5','sqftmain2','sqftmain3','sqftmain1', 'sqftmain4', 'sqftmain5','units1', 'units3', 'units2', 'units5', 'units4','bathrooms5', 'bathrooms4', 'bathrooms1','bathrooms3', 'bathrooms2']]= parcelsdf[['bedrooms1', 'bedrooms2', 'bedrooms3','bedrooms4', 'bedrooms5','sqftmain2','sqftmain3','sqftmain1', 'sqftmain4', 'sqftmain5','units1', 'units3', 'units2', 'units5', 'units4','bathrooms5', 'bathrooms4', 'bathrooms1','bathrooms3', 'bathrooms2']].fillna(0)

In [11]:
parcelsdf[['bedrooms1', 'bedrooms2', 'bedrooms3','bedrooms4', 'bedrooms5','sqftmain2','sqftmain3','sqftmain1', 'sqftmain4', 'sqftmain5','units1', 'units3', 'units2', 'units5', 'units4','bathrooms5', 'bathrooms4', 'bathrooms1','bathrooms3', 'bathrooms2']]= parcelsdf[['bedrooms1', 'bedrooms2', 'bedrooms3','bedrooms4', 'bedrooms5','sqftmain2','sqftmain3','sqftmain1', 'sqftmain4', 'sqftmain5','units1', 'units3', 'units2', 'units5', 'units4','bathrooms5', 'bathrooms4', 'bathrooms1','bathrooms3', 'bathrooms2']].astype(int)

In [12]:
parcelsdf['units_total']=parcelsdf['units1']+parcelsdf['units2']+parcelsdf['units3']+parcelsdf['units4']+parcelsdf['units5']

parcelsdf['bedrooms_total']=parcelsdf['bedrooms1']+parcelsdf['bedrooms2']+parcelsdf['bedrooms3']+parcelsdf['bedrooms4']+parcelsdf['bedrooms5']

parcelsdf['bathrooms_total']=parcelsdf['bathrooms1']+parcelsdf['bathrooms2']+parcelsdf['bathrooms3']+parcelsdf['bathrooms4']+parcelsdf['bathrooms5']

parcelsdf['sqft_total']=parcelsdf['sqftmain1']+parcelsdf['sqftmain2']+parcelsdf['sqftmain3']+parcelsdf['sqftmain4']+parcelsdf['sqftmain5']



We also need to find the average built age of the structures on the parcel. However some are inccorectly recorded as 0, and we will need to convert those to null first.

In [13]:
import numpy as np

In [14]:
parcelsdf[['yearbuilt5', 'yearbuilt4', 'yearbuilt1','yearbuilt2','yearbuilt3']]=parcelsdf[['yearbuilt5', 'yearbuilt4', 'yearbuilt1','yearbuilt2','yearbuilt3']].replace(0,np.nan)

In [15]:
parcelsdf['yearbuilt_avg']=parcelsdf[['yearbuilt5', 'yearbuilt4', 'yearbuilt1','yearbuilt2','yearbuilt3']].mean(axis=1)

parcelsdf['yearbuilt_avg']=parcelsdf['yearbuilt_avg'].astype(int)

print(parcelsdf.yearbuilt_avg.unique())

[1961 1948 1981 1926 1952 1950 1977 1956 1980 1959 1954 1973 1945 1901
 1912 1908 1933 1895 1906 1913 1935 1890 1914 1921 1931 1922 1898 1905
 1927 1923 1953 1904 1909 1900 1911 1929 1885 1936 1919 2003 1907 1893
 1944 1941 1902 1880 1887 1899 1940 2006 1896 1918 1947 1892 1963 1897
 1910 1978 1903 2001 1966 1946 1939 1975 1938 1951 1924 1915 2004 1937
 1920 1960 1965 2007 1986 2002 1964 1958 2010 1916 1979 1962 1957 1990
 1932 2009 1997 1991 1930 1993 1992 1928 2011 1985 1943 1949 1988 1925
 1976 1917 1891 2016 1942 2008 2014 1999 1881 1934 1984 1968 1989 1884
 1972 1971 1967 1983 1994 1955 1969 2005 1982 1987 1998 1996 2017 1894
 1995 1889 2015 1970 1974 1888 2013 2000 2012 2018 1886 1882 1804 1883
 2019 1833 1875 1867 1857 1819 1866 1870 1861 1827 1815 1828 1806 1879
 1873 1864 1834 1863 1872 1852 1877 1868 1808 1869 1865 1855 1812 1825
 1859]


Now we can generate some more interesting parcel characteristics, in terms of built area to parcel area, and estimated density.

In [16]:
parcelsdf['built_to_land']=parcelsdf['sqft_total']/parcelsdf['shape.starea()']

parcelsdf['estimated_density_per_acre']=(parcelsdf['units_total']/(parcelsdf['shape.starea()']/43560))

parcelsdf.columns

Index(['fid', 'center_lon', 'yearbuilt5', 'yearbuilt4', 'yearbuilt1',
       'usetype', 'yearbuilt2', 'apn', 'bedrooms1', 'bedrooms2', 'bedrooms3',
       'bedrooms4', 'bedrooms5', 'usedescription', 'mailaddress', 'sqftmain2',
       'sqftmain3', 'sqftmain1', 'sqftmain4', 'sqftmain5', 'mailcity',
       'units1', 'units3', 'units2', 'units5', 'units4', 'ownerfullname',
       'agencyname', 'ain', 'bathrooms5', 'bathrooms4', 'bathrooms1',
       'bathrooms3', 'bathrooms2', 'situsfulladdress', 'shape.starea()',
       'shape.stlength()', 'yearbuilt3', 'center_lat', 'ls1_sale_amount',
       'ls1_sale_date', 'ls2_sale_amount', 'ls2_sale_date', 'units_total',
       'bedrooms_total', 'bathrooms_total', 'sqft_total', 'yearbuilt_avg',
       'built_to_land', 'estimated_density_per_acre'],
      dtype='object')

Now, we can indentify parcels that were built or sold in the years in question

In [17]:
print(parcelsdf['ls1_sale_date'].unique())

[19741018.        0. 20191007. ... 19890527. 19721228. 20170422.]


In [18]:
#what happened here- how can I fix this without dropping the rows with 0?
parcelsdf['ls1_sale_date']=parcelsdf.ls1_sale_date.fillna(0)
parcelsdf['ls1_sale_date']=parcelsdf.ls1_sale_date.astype(int).astype(str)
parcelsdf['last_sold_date']=pd.to_datetime(parcelsdf['ls1_sale_date'],infer_datetime_format=True,errors='coerce')
print(parcelsdf.last_sold_date.unique())

['1974-10-18T00:00:00.000000000'                           'NaT'
 '2019-10-07T00:00:00.000000000' ... '1989-05-27T00:00:00.000000000'
 '1972-12-28T00:00:00.000000000' '2017-04-22T00:00:00.000000000']


In [19]:
parcelsdf2=parcelsdf[pd.isna(parcelsdf['last_sold_date'])]
print(len(parcelsdf2))

32235


In [20]:
parcelsdf['ls1_sale_date']=parcelsdf.ls1_sale_date.astype(int)
parcelsdf['sold_14_19']=parcelsdf.ls1_sale_date.apply(lambda x:1 if (x>20150000 and x<20200000) else 0)

sold_14_19df=parcelsdf[parcelsdf['sold_14_19']==1]

print(len(sold_14_19df))

147551


In [21]:
parcelsdf['built_14_19']=parcelsdf.yearbuilt_avg.apply(lambda x: 1 if (x>2014 and x<2020) else 0)

built_14_19df=parcelsdf[parcelsdf['built_14_19']==1]

print(len(built_14_19df))

8688


In [22]:
#ask about this too for text, esp wildcard characters

parcelsdf['ownerfullname']=parcelsdf['ownerfullname'].astype(str)

def process_name(parcelsdf):
    if pd.notna(parcelsdf['agencyname']):
        return 0
    elif  ',' in parcelsdf['ownerfullname'] and 'TRS' not in parcelsdf['ownerfullname'] and 'TRUST' not in parcelsdf['ownerfullname'] and 'TR' not in parcelsdf['ownerfullname'] and 'GP' not in parcelsdf['ownerfullname'] and 'LP' not in parcelsdf['ownerfullname'] and 'LLP' not in parcelsdf['ownerfullname'] and 'PARTNERS' not in parcelsdf['ownerfullname'] and 'LLC' not in parcelsdf['ownerfullname'] and 'LIMITED' not in parcelsdf['ownerfullname'] and 'LTD' not in parcelsdf['ownerfullname'] and 'PARTNERSHIP' not in parcelsdf['ownerfullname']:
        return 1
    elif 'TRS' in parcelsdf['ownerfullname'] or 'TRUST' in parcelsdf['ownerfullname'] or 'TR' in parcelsdf['ownerfullname']:
        return 2
    elif 'LP' in parcelsdf['ownerfullname'] or 'GP'in parcelsdf['ownerfullname'] or 'PARTNERS'in parcelsdf['ownerfullname'] or 'PARTNER'in parcelsdf['ownerfullname'] or 'PARTNERSHIP'in parcelsdf['ownerfullname'] :
        return 3
    elif 'LLC' in parcelsdf['ownerfullname'] or 'LLP'in parcelsdf['ownerfullname'] or 'LTD' in parcelsdf['ownerfullname'] or'LIMITED'in parcelsdf['ownerfullname']:
        return 4
    else: 
        return 5
    
parcelsdf['landlord_type']=parcelsdf.apply(process_name, axis=1)

print(parcelsdf.landlord_type.unique())

[2 1 5 4 3 0]


In [23]:
parcelsdf.landlord_type.value_counts()

1    416397
2    232633
4     36027
5      9971
3      8752
0      1454
Name: landlord_type, dtype: int64

In [24]:
#how to create a new df of ll size by counting units and counts of mailing address, then apply to original df

tempdf1=parcelsdf.groupby('mailaddress')['mailaddress'].count()

tempdf2=parcelsdf.groupby('mailaddress')['units_total'].sum()

llaggdf = pd.DataFrame({'n_ll_properties': tempdf1, 'n_ll_units': tempdf2})

In [25]:
print(llaggdf.head(10))

                                   n_ll_properties  n_ll_units
mailaddress                                                   
 1 16 7 SAIWAI CHO SHIKI SHI                     1          23
 1 RUE DU MOULIN RENARD                          1           1
 1-10-4 KAMINOGE SETAGAYA KU                     1           5
 1-14-5 YAKUIN-CHUO-KU                           1           1
 1-20-23 SAKAI MUSASHIMO-SHI                     1           1
 1-8-5 HARUMI NO1706 CHOU KU                     1           1
 10 JALAN 13/6 GREENVILLE                        1           1
 10/F SURSON COMM BLVD 142 AUSTIN                1           1
 100701 RED RYDER RD                             1           1
 104-375 WATER ST VANCOUVER                      1           4


In [26]:
parcelsdf=parcelsdf.join(llaggdf, how='left', on='mailaddress')

parcelsdf.head(10)


,fid,center_lon,yearbuilt5,yearbuilt4,yearbuilt1,usetype,yearbuilt2,apn,bedrooms1,bedrooms2,...,sqft_total,yearbuilt_avg,built_to_land,estimated_density_per_acre,last_sold_date,sold_14_19,built_14_19,landlord_type,n_ll_properties,n_ll_units
0,5,-118.434714,NaN,NaN,1961.0,Residential,NaN,4357-015-017,3,0,...,2370,1961,0.173059,3.180789,1974-10-18,0,0,2,1.0,1.0
1,7,-118.509913,NaN,NaN,1948.0,Residential,NaN,2690-001-025,3,0,...,1969,1948,0.071572,1.583370,NaT,0,0,2,3.0,3.0
2,8,-118.434041,NaN,NaN,1981.0,Residential,NaN,4357-009-014,4,0,...,6180,1981,0.032764,0.230936,2019-10-07,1,0,2,4.0,12.0
3,21,-118.437185,NaN,NaN,1926.0,Residential,NaN,4357-011-006,7,0,...,6406,1926,0.038333,0.260656,1996-07-15,0,0,1,1.0,1.0
4,44,-118.511595,NaN,NaN,1952.0,Residential,NaN,2768-018-013,4,0,...,2773,1952,0.146416,2.299997,1999-10-28,0,0,2,1.0,1.0
5,47,-118.510880,NaN,NaN,1950.0,Residential,NaN,2690-003-012,3,0,...,2369,1950,0.079438,1.460660,1997-09-25,0,0,2,3.0,3.0
6,58,-118.435081,NaN,NaN,1977.0,Residential,NaN,4357-015-006,3,0,...,2644,1977,0.046179,0.760803,2003-08-08,0,0,1,1.0,1.0
7,64,-118.511546,NaN,NaN,1956.0,Residential,NaN,2766-012-018,5,0,...,3576,1956,0.176879,2.154601,1997-10-03,0,0,2,1.0,1.0
8,102,-118.511540,NaN,NaN,1961.0,Residential,NaN,2768-022-022,5,0,...,2981,1961,0.190678,2.786298,NaT,0,0,2,1.0,1.0
9,122,-118.437277,NaN,NaN,1961.0,Residential,NaN,4357-004-018,2,0,...,1709,1961,0.113921,2.903675,1990-05-24,0,0,5,3.0,3.0


In [27]:
#We also need a bool indicator of the corporate status for regression

parcelsdf['landlord_iscorp']=parcelsdf.landlord_type.apply(lambda x:1 if x>2 else 0)

parcelsdf.landlord_iscorp.value_counts()

0    650484
1     54750
Name: landlord_iscorp, dtype: int64

Now, we will turn to assigning neighborhood characteristics for the parcels in question, using census data from the ACS acquired through cenpy's API tool. Index of tables https://www2.census.gov/programs-surveys/acs/tech_docs/table_shells/2019/

In [28]:
import requests
import cenpy
from cenpy import products

acs19=cenpy.products.ACS(2019)

In [29]:
vars_to_download_19 = {
    "B25064_001E": "median_rent_2019",
    "B25077_001E": "median_value_2019",
    "B03002_001E": "total_pop_2019",
    "B03002_003E": "white_2019",
    "B03002_004E": "black_2019",
    "B03002_012E": "latx_2019",
    "B25003_001E": "housing_occupied_2019",
    "B25003_003E": "housing_rented_2019",
    "B09019_007E": "hh_female_head_2019",
    "B09019_001E": "hh_total_2019",
    "B15003_001E": "educ_all_2019",
    "B15003_015E": "educ_bachelors_2019",
    "B15003_016E": "educ_masters_2019",
    "B15003_017E": "educ_professional_2019",
    "B15003_018E": "educ_doctoral_2019", 
    "B19013_001E": "median_income_2019",
    "B23025_003E": "civ_laborforce_2019",
    "B23025_005E": "civ_unemployed_2019",
    "B17001_001E": "pov_all_2019",
    "B17001_002E": "pov_extreme_2019",
    "B17001_003E": "pov_severe_2019",
    "B17001_004E": "pov_in_2019",
    "B11005_001E": "hh_kids_all_2019",
    "B11005_002E": "hh_kids_yes_2019",
    "B25002_001E": "units_all_2019",
    "B25002_003E": "units_vacant_2019"
    
}
vars_to_download_l_19 = list(vars_to_download_19.keys())

In [ ]:
tracts19gdf=products.ACS(2019).from_county('Los Angeles, CA',level='tract',variables=vars_to_download_l_19)

/Applications/anaconda3/envs/UP299/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
print(tracts19gdf.head(10))

In [ ]:
tracts19gdf=tracts19gdf.rename(columns=vars_to_download_19)
print(tracts19gdf.columns.unique())

In [ ]:
tracts19gdf.columns
columns19 = ['total_pop_2019', 'white_2019', 'black_2019',
       'latx_2019', 'hh_total_2019', 'hh_female_head_2019', 'hh_kids_all_2019',
       'hh_kids_yes_2019', 'educ_all_2019', 'educ_bachelors_2019',
       'educ_masters_2019', 'educ_professional_2019', 'educ_doctoral_2019',
       'pov_all_2019', 'pov_extreme_2019', 'pov_severe_2019', 'pov_in_2019',
       'median_income_2019', 'civ_laborforce_2019', 'civ_unemployed_2019',
       'units_all_2019', 'units_vacant_2019', 'housing_occupied_2019',
       'housing_rented_2019', 'median_rent_2019', 'median_value_2019']
tracts19gdf[tracts19gdf[columns19] < 0]=np.nan
tracts19gdf.head(10)

In [ ]:
tracts19gdf.dtypes

In [ ]:
tracts19gdf['part_black_2019']=(tracts19gdf['black_2019']/tracts19gdf['total_pop_2019'])*100
tracts19gdf['part_latx_2019']=(tracts19gdf['latx_2019']/tracts19gdf['total_pop_2019'])*100
tracts19gdf['part_white_2019']=(tracts19gdf['white_2019']/tracts19gdf['total_pop_2019'])*100
tracts19gdf['part_female_head_2019']=(tracts19gdf['hh_female_head_2019']/tracts19gdf['hh_total_2019'])*100
tracts19gdf['part_hh_kids_2019']=(tracts19gdf['hh_kids_yes_2019']/tracts19gdf['hh_kids_all_2019'])*100
tracts19gdf['part_rented_2019']=(tracts19gdf['housing_rented_2019']/tracts19gdf['housing_occupied_2019'])*100
tracts19gdf['part_unemployed_2019']=(tracts19gdf['civ_unemployed_2019']/tracts19gdf['civ_laborforce_2019'])*100
tracts19gdf['part_poverty_2019']=((tracts19gdf['pov_in_2019']+tracts19gdf['pov_severe_2019']+tracts19gdf['pov_extreme_2019'])/tracts19gdf['pov_all_2019'])*100
tracts19gdf['part_vacant_2019']=(tracts19gdf['units_vacant_2019']/tracts19gdf['units_all_2019'])*100
tracts19gdf['part_college_2019']=((tracts19gdf['educ_bachelors_2019']+tracts19gdf['educ_masters_2019']+tracts19gdf['educ_doctoral_2019']+tracts19gdf['educ_professional_2019'])/tracts19gdf['educ_all_2019'])*100

tracts19gdf.head(10)


Beacuse cenpy does not offer a prepackaged ACS 14 prodcut, we used their remote api connection query framework to download the same data for LA County tracts for that year, to be joined via tract ID. Variables identified here https://api.census.gov/data/2014/acs/acs5/variables.html.

In [ ]:
vars_to_download_14 = {
    "B25064_001E": "median_rent_2014",
    "B25077_001E": "median_value_2014",
    "B03002_001E": "total_pop_2014",
    "B03002_003E": "white_2014",
    "B03002_004E": "black_2014",
    "B03002_012E": "latx_2014",
    "B25003_001E": "housing_occupied_2014",
    "B25003_003E": "housing_rented_2014",
    "B09019_007E": "hh_female_head_2014",
    "B09019_001E": "hh_total_2014",
    "B15003_001E": "educ_all_2014",
    "B15003_015E": "educ_bachelors_2014",
    "B15003_016E": "educ_masters_2014",
    "B15003_017E": "educ_professional_2014",
    "B15003_018E": "educ_doctoral_2014", 
    "B19013_001E": "median_income_2014",
    "B23025_003E": "civ_laborforce_2014",
    "B23025_005E": "civ_unemployed_2014",
    "B17001_001E": "pov_all_2014",
    "B17001_002E": "pov_extreme_2014",
    "B17001_003E": "pov_severe_2014",
    "B17001_004E": "pov_in_2014",
    "B11005_001E": "hh_kids_all_2014",
    "B11005_002E": "hh_kids_yes_2014",
    "B25002_001E": "units_all_2014",
    "B25002_003E": "units_vacant_2014"
    
}
vars_to_download_l_14 = list(vars_to_download_14.keys())

In [ ]:
alist=list(cenpy.explorer.available().index.unique())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(alist)

In [ ]:
conn=cenpy.remote.APIConnection('ACSDT5Y2014')

In [ ]:
tracts14df=cenpy.remote.APIConnection.query(conn,cols=vars_to_download_l_14, geo_unit='tract:*', geo_filter={'state':'06','county':'037'})

In [ ]:
tracts14df.head()

In [ ]:
tracts14df=tracts14df.rename(columns=vars_to_download_14)
tracts14df.head()

In [ ]:
tracts14df=tracts14df.astype(int)
tracts14df.dtypes

In [ ]:
tracts14df.columns

In [ ]:
import numpy as np

In [ ]:
columns14 = ['median_rent_2014', 'median_value_2014', 'total_pop_2014', 'white_2014',
       'black_2014', 'latx_2014', 'housing_occupied_2014',
       'housing_rented_2014', 'hh_female_head_2014', 'hh_total_2014',
       'educ_all_2014', 'educ_bachelors_2014', 'educ_masters_2014',
       'educ_professional_2014', 'educ_doctoral_2014', 'median_income_2014',
       'civ_laborforce_2014', 'civ_unemployed_2014', 'pov_all_2014',
       'pov_extreme_2014', 'pov_severe_2014', 'pov_in_2014',
       'hh_kids_all_2014', 'hh_kids_yes_2014', 'units_all_2014',
       'units_vacant_2014']
tracts14df[tracts14df[columns14] < 0]=np.nan

In [ ]:
tracts14df.head(10)

In [ ]:
tracts14df['part_black_2014']=(tracts14df['black_2014']/tracts14df['total_pop_2014'])*100
tracts14df['part_latx_2014']=(tracts14df['latx_2014']/tracts14df['total_pop_2014'])*100
tracts14df['part_white_2014']=(tracts14df['white_2014']/tracts14df['total_pop_2014'])*100
tracts14df['part_female_head_2014']=(tracts14df['hh_female_head_2014']/tracts14df['hh_total_2014'])*100
tracts14df['part_hh_kids_2014']=(tracts14df['hh_kids_yes_2014']/tracts14df['hh_kids_all_2014'])*100
tracts14df['part_rented_2014']=(tracts14df['housing_rented_2014']/tracts14df['housing_occupied_2014'])*100
tracts14df['part_unemployed_2014']=(tracts14df['civ_unemployed_2014']/tracts14df['civ_laborforce_2014'])*100
tracts14df['part_poverty_2014']=((tracts14df['pov_in_2014']+tracts14df['pov_severe_2014']+tracts14df['pov_extreme_2014'])/tracts14df['pov_all_2014'])*100
tracts14df['part_vacant_2014']=(tracts14df['units_vacant_2014']/tracts14df['units_all_2014'])*100
tracts14df['part_college_2014']=((tracts14df['educ_bachelors_2014']+tracts14df['educ_masters_2014']+tracts14df['educ_doctoral_2014']+tracts14df['educ_professional_2014'])/tracts14df['educ_all_2014'])*100

tracts14df.head(10)

In [ ]:
tracts19gdf['tract']=tracts19gdf['tract'].astype(int)
tracts19gdf.dtypes

In [ ]:
tracts19gdf=tracts19gdf.set_index('tract')
tracts14df=tracts14df.set_index('tract')

joinedtractsgdf=tracts19gdf.join(tracts14df, on='tract', how='left', rsuffix='_14')

#joinedtractsgdf[joinedtractsgdf < 0]=np.nan

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(joinedtractsgdf.columns)


In [ ]:
joinedtractsgdf=joinedtractsgdf.fillna(joinedtractsgdf.mean())

joinedtractsgdf.head(10)

In [ ]:
joinedtractsgdf.dtypes

In [ ]:
joinedtractsgdf['change_rent_14_19']=joinedtractsgdf['median_rent_2019']-joinedtractsgdf['median_rent_2014']
joinedtractsgdf['change_value_14_19']=joinedtractsgdf['median_value_2019']-joinedtractsgdf['median_value_2014']
joinedtractsgdf['change_part_rented_14_19']=joinedtractsgdf['part_rented_2019']-joinedtractsgdf['part_rented_2014']
joinedtractsgdf['change_part_white_14_19']=joinedtractsgdf['part_white_2019']-joinedtractsgdf['part_white_2014']
joinedtractsgdf['change_part_black_14_19']=joinedtractsgdf['part_black_2019']-joinedtractsgdf['part_black_2014']
joinedtractsgdf['change_part_latx_14_19']=joinedtractsgdf['part_latx_2019']-joinedtractsgdf['part_latx_2014']
joinedtractsgdf['change_part_college_14_19']=joinedtractsgdf['part_college_2019']-joinedtractsgdf['part_college_2014']
joinedtractsgdf['change_part_unemployed_14_19']=joinedtractsgdf['part_unemployed_2019']-joinedtractsgdf['part_unemployed_2014']
joinedtractsgdf['change_part_hh_kids_14_19']=joinedtractsgdf['part_hh_kids_2019']-joinedtractsgdf['part_hh_kids_2014']
joinedtractsgdf['change_part_female_head_14_19']=joinedtractsgdf['part_female_head_2019']-joinedtractsgdf['part_female_head_2014']
joinedtractsgdf['change_part_poverty_14_19']=joinedtractsgdf['part_poverty_2019']-joinedtractsgdf['part_poverty_2014']
joinedtractsgdf['change_part_vacant_14_19']=joinedtractsgdf['part_vacant_2019']-joinedtractsgdf['part_vacant_2014']
joinedtractsgdf.head(10)

In [ ]:
joinedtractsgdf=joinedtractsgdf.reset_index()
joinedtractsgdf.head(10)

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

%matplotlib inline

fig, ax=plt.subplots(figsize=(20,20))

joinedtractsgdf.plot(ax=ax, legend=True)

plt.title('LA Census Tracts')

ctx.add_basemap(ax,source=ctx.providers.Stamen.TonerLite,zoom=11)

ax.set_xticks([])
ax.set_yticks([])

Next, we will join the tract level information to the parcel scale, and generate the relevant proximity metrics.

In [ ]:
parcelsgdf = gpd.GeoDataFrame(parcelsdf, geometry=gpd.points_from_xy(parcelsdf.center_lon, parcelsdf.center_lat))

In [ ]:
parcelsgdf = parcelsgdf.set_crs("EPSG:4326")

parcelsgdf = parcelsgdf.to_crs("EPSG:3857")

print(parcelsgdf.crs)

In [ ]:
#sparcelsgdf is Ellis-ed parcels (parcelsgdf) + [all parcels in LA + census data/demographic predictors (joinedtractsgdf)]
sparcelsgdf = gpd.sjoin(parcelsgdf, joinedtractsgdf, how="left", op='intersects',rsuffix="_tract")
sparcelsgdf.head()

In [ ]:
sparcelsgdf.columns

In [ ]:
fig, ax=plt.subplots(figsize=(20,20))

sparcelsgdf.plot(ax=ax, legend=True)

plt.title('LA Census Tracts')

ctx.add_basemap(ax,source=ctx.providers.Stamen.TonerLite,zoom=11)

ax.set_xticks([])
ax.set_yticks([])

In [ ]:
#sparcelsgdf['geometry']
#sparcels geometry is point data of parcels, not shape data of census tracts

In [ ]:
# Save sparcelsgdf to file
#sparcelsgdf.to_csv("sparcelsgdf.csv")

In [ ]:
built_14_19gdf=gpd.GeoDataFrame(built_14_19df, geometry=gpd.points_from_xy(built_14_19df.center_lon, built_14_19df.center_lat))
sold_14_19gdf=gpd.GeoDataFrame(sold_14_19df, geometry=gpd.points_from_xy(sold_14_19df.center_lon, sold_14_19df.center_lat))

built_14_19gdf = built_14_19gdf.set_crs("EPSG:4326")
built_14_19gdf = built_14_19gdf.to_crs("EPSG:3857")

sold_14_19gdf = sold_14_19gdf.set_crs("EPSG:4326")
sold_14_19gdf = sold_14_19gdf.to_crs("EPSG:3857")



In [ ]:
#built_14_19gdf.to_csv("built_14_19gdf.csv")
#sold_14_19gdf.to_csv("sold_14_19gdf.csv")

In [ ]:
sparcelsgdf1=sparcelsgdf[pd.isna(sparcelsgdf['hh_total_2019'])]
print(len(sparcelsgdf1))

In order to identify proximity to high rent tracts and fast increasing rent tracts, we identified the 75th percentile value for 2019 rents and change of rents 2014-2019 to create new layers for a distance analysis.

In [ ]:
joinedtractsgdf['median_rent_2019'].describe()

In [ ]:
highrenttractsgdf=joinedtractsgdf[joinedtractsgdf['median_rent_2019']>=1821.750000]

In [ ]:
joinedtractsgdf['change_rent_14_19'].describe()

In [ ]:
highchangetractsgdf=joinedtractsgdf[sparcelsgdf['change_rent_14_19']>=355.00]

### Calculating percent of parcels bought and sold in each tract:

In [ ]:
built_14_19gdf.head(5)

In [ ]:
joinedtractsbuilt = gpd.sjoin(built_14_19gdf, joinedtractsgdf, how="left", op='intersects')
joinedtractsbuilt.head()

In [ ]:
joinedtractssold = gpd.sjoin(sold_14_19gdf, joinedtractsgdf, how="left", op='intersects')
joinedtractssold.head()

In [ ]:
built1419 = joinedtractsbuilt.groupby('tract')['built_14_19'].apply(lambda x: x[x == True].count())

In [ ]:
sold1419 = joinedtractssold.groupby('tract')['sold_14_19'].apply(lambda x: x[x == True].count())

In [ ]:
joinedtractsgdf = joinedtractsgdf.set_index('tract')
joinedtractsgdf

In [ ]:
aggDf = pd.DataFrame({'n_built1419': built1419, 'n_sold1419': sold1419})

In [ ]:
joinedbuiltsold = pd.merge(joinedtractsgdf,aggDf, how='left', on='tract')
joinedbuiltsold

In [ ]:
totalparcels = sparcelsgdf.groupby('tract').size()
totalparcels

In [ ]:
joinedbuiltsold['totalparcels'] = totalparcels
joinedbuiltsold.head(5)

In [ ]:
joinedbuiltsold['pctbuilt'] = joinedbuiltsold['n_built1419']/joinedbuiltsold['totalparcels']*100
joinedbuiltsold['pctsold'] = joinedbuiltsold['n_sold1419']/joinedbuiltsold['totalparcels']*100
joinedbuiltsold.head(5)

In [ ]:
joinedbuiltsold = joinedbuiltsold.fillna(0)
joinedbuiltsold.head(5)

In [ ]:
sparcelsgdf.set_index('ain', inplace=True)

In [ ]:
sparcelsgdf['ain']

In [ ]:
def get_closest_tract(row):
    parcel = row.geometry
    distances = highrenttractsgdf.distance(parcel)
    min_dist = distances.min()
    return pd.Series([min_dist])

sparcelsgdf[['dist_to_nearest_highrenttract']] = sparcelsgdf.apply(get_closest_tract, axis=1)
sparcelsgdf.head()


In [ ]:
def get_closest_tract_2(row):
    parcel2 = row.geometry
    distances2 = highchangetractsgdf.distance(parcel)
    min_dist2 = distances.min()
    return pd.Series([min_dist2])

sparcelsgdf[['dist_to_nearest_highchangetract']] = sparcelsgdf.apply(get_closest_tract_2, axis=1)
sparcelsgdf.head()


Now we want to create our datasets for the statistical and clustering tests, one of ellis properties with the parcel information joined, and another of all parcels, with an indication of the ellis status.

In [ ]:
studyellisdf.dtypes

In [ ]:
studyellisdf=studyellisdf.set_index('ain')
sparcelgsdf=sparcelsgdf.set_index('ain')

parcelsjoindf=sparcelsgdf.join(studyellisdf, on='ain', how='left', rsuffix='_ellis')

parcelsjoindf.head()

In [ ]:
parcelsjoindf1=parcelsjoindf[pd.isna(parcelsjoindf['units_withdrawn'])]
print(len(parcelsjoindf1))

In [ ]:
parcelsjoindf2=parcelsjoindf[pd.notna(parcelsjoindf['units_withdrawn'])]
print(len(parcelsjoindf2))

In [ ]:
parcelsjoindf3=parcelsjoindf[parcelsjoindf['units_total']>1]

parcelsjoindf4=parcelsjoindf3[parcelsjoindf3['yearbuilt_avg']<1979]

print(len(parcelsjoindf4))

In [ ]:
parcelsjoindf5=parcelsjoindf4[pd.notna(parcelsjoindf4['units_withdrawn'])]

print(len(parcelsjoindf5))

In [ ]:
ellisjoindf = studyellisdf.join(parcelsdf, on='ain', how='left', rsuffix='parcel')

ellisjoindf.head()

With our Ellis datset, we will also want to create a new field that takes the difference between the filing date and the last sale date for the property. Thereby we can filter out negative diffs (landlord no longer owns property) and study the difference in neighborhood conditions in changing filing times.

In [ ]:
ellisjoindf['time_to_file']=ellisjoindf['application_date']-ellisjoindf['last_sold_date']
ellisjoindf['time_to_file']=ellisjoindf['time_to_file'].astype('timedelta64[D]')

In [ ]:
ellisjoindf.head()

In [ ]:
ellisjoindf.dtypes

Now we'll want to check and see to waht extent parcels have been built since an Ellis Filing, and to what extent they have been sold since the filing, which will determine whether or not the record can remain in the sample. 

In [ ]:
ellisjoindf.yearbuilt_avg.unique()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(ellisjoindf.time_to_file.value_counts())

In [ ]:
print(len(ellisjoindf))

In [ ]:
ellisjoindf1=ellisjoindf[ellisjoindf['time_to_file']>-1]
print(len(ellisjoindf1))

In [ ]:
ellisjoindf2=ellisjoindf1[ellisjoindf1['landlord_type']>0]
print(len(ellisjoindf2))